In [ ]:
import os
import sys
import pandas as pd

sys.path.append('..')

from tqdm import tqdm
import time
from unstructured.staging.base import elements_from_json
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from sentence_transformers import SentenceTransformer, util
from langchain.embeddings.openai import OpenAIEmbeddings

os.environ["OPENAI_API_KEY"] = "сюда"
os.environ["OPENAI_API_BASE"] = "сюда"

import pandas as pd

from src.utils.paths import get_project_path
from openai import OpenAI

In [4]:
def load_vector_db(load_path="./db_BAAI_bge-m3"):
    vector_store = FAISS.load_local(
        load_path, HuggingFaceEmbeddings(model_name='BAAI/bge-m3'),
        allow_dangerous_deserialization=True
    )
    print(f"Индекс загружен из директории: {load_path}")
    return vector_store


def get_rag_response(ques, retriever):
    """
    Generates an answer using RAG and OpenAI API.
    """
    retrieved_docs = retriever.get_relevant_documents(ques)
    context = "\n\n".join(doc.page_content for doc in retrieved_docs)

    llm = OpenAI(base_url=os.environ["OPENAI_API_BASE"])

    response_big = llm.chat.completions.create(
        model="openai/gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a highly intelligent assistant who helps find and explain information."
            },
            {
                "role": "user",
                "content": f"You have access to the following contextual information: {context} Using this information, answer the following question clearly, in detail, and professionally: Question: {ques} If the information in the context is insufficient, acknowledge it honestly and suggest a logical next step. Answer in Russian."
            },
        ],
        temperature=0.5,
        max_tokens=3000,
    )

    return response_big.choices[0].message.content.strip()


def sequential_process_rag_responses(data, retriever):
    """
    Process RAG responses sequentially with rate limiting.
    """
    for idx, row in tqdm(data.iterrows(), total=len(data), desc="Processing RAG responses"):
        try:
            data.at[idx, 'RAG_Ответ'] = get_rag_response(row['Вопрос'], retriever)
            time.sleep(1.1)  # Wait for 1.1 seconds to respect the rate limit
        except Exception as e:
            print(f"Error processing question {idx}: {e}")


def evaluate_similarity(data):
    similarities = []
    for idx, row in data.iterrows():
        reference = row['Правильный ответ']
        prediction = row['RAG_Ответ']
        embeddings = model.encode([reference, prediction], convert_to_tensor=True)
        similarity = util.cos_sim(embeddings[0], embeddings[1]).item()
        similarities.append(similarity)
    return similarities

In [ ]:
# Загружаем БД и передаем ее в третриве

db = load_vector_db()

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

/tmp/ipykernel_3455655/4262839051.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  load_path, HuggingFaceEmbeddings(model_name='BAAI/bge-m3'),


Индекс загружен из директории: ./db_BAAI_bge-m3


In [ ]:
# получаем ответ из валидационных вопросов

data = pd.read_csv(os.path.join(get_project_path(), 'data', 'Answer_Promt.csv'))[['Вопрос', 'Правильный ответ']].iloc[:5]

sequential_process_rag_responses(data, retriever)

Processing RAG responses:   0%|          | 0/106 [00:00<?, ?it/s]/tmp/ipykernel_3455655/4262839051.py:14: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(ques)


Processing RAG responses:  69%|██████▉   | 73/106 [18:07<09:24, 17.11s/it]

Error processing question 72: Connection error.


Processing RAG responses:  98%|█████████▊| 104/106 [26:21<00:30, 15.22s/it]

Error processing question 103: Error code: 429 - {'error': {'message': 'Rate-limit error: You send more than 1 request per 1.0 second. Try later.', 'code': 429}}


Processing RAG responses: 100%|██████████| 106/106 [27:00<00:00, 15.28s/it]


In [8]:
model = SentenceTransformer('BAAI/bge-m3')

data['Семантическая_Близость'] = evaluate_similarity(data)

average_similarity = data['Семантическая_Близость'].dropna().mean()
print(f"Средняя семантическая близость: {average_similarity:.2f}")

Средняя семантическая близость: 0.71
